# Fly-tipping Report

This task requires using the UK government"s [fly-tipping dataset](https://www.data.gov.uk/dataset/1388104c-3599-4cd2-abb5-ca8ddeeb4c9c/fly-tipping-in-england) to investigate the following questions:

- Which kind of incidents happen most in your region?
- How many people have been sent to prison in your region?
- In which region is flytipping growing the fastest?
- Which region has the biggest fluctuations in clean costs?
- Which region is losing the most money from flytipping?

In [21]:
import urllib.request
import os

FLYTIPPING_ACTIONS_URL = "https://s3.eu-west-1.amazonaws.com/data.defra.gov.uk/statistics_2021/LA_flytipping_actions_2012-13_to_2020-21.csv"
FLYTIPPING_INCIDENTS_URL = "https://s3.eu-west-1.amazonaws.com/data.defra.gov.uk/statistics_2021/LA_flytipping_incidents_2012-13_to_2020-21_updated.csv"

def is_flytipping_data_downloaded(incidents_file_name="flytipping-incidents.csv", actions_file_name="flytipping-actions.csv"):
    """Check if the required datasets are downloaded in the local directory, 
    if not downloads them.
    """
    if not os.path.isfile(incidents_file_name) and not os.path.isfile(actions_file_name):
        urllib.request.urlretrieve(FLYTIPPING_INCIDENTS_URL, incidents_file_name)
        urllib.request.urlretrieve(FLYTIPPING_ACTIONS_URL, actions_file_name)
    else:
        return "Files already in local directory"
    

is_flytipping_data_downloaded()

'Files already in local directory'

In [22]:
import pandas as pd

df_incidents = pd.read_csv("./flytipping-incidents.csv", encoding="ISO-8859-1", header=1)

df_actions = pd.read_csv("./flytipping-actions.csv", encoding="ISO-8859-1", header=1)

## Clean data

- Check for missing values and decide whether to leave within the dataset or remove them
- Cast the correct data types onto columns

### Check for null values

- See if the number of null values will affect the quality of the data

In [23]:
def sum_null_values(df):
    """Sums the number of null valued within a column in 
    a dataframe

    Args:
        df: Pandas Dataframe
    """
    missing_data = df.isnull()
    missing_data_count = missing_data.sum()
    
    return missing_data_count


In [24]:
sum_null_values(df_incidents)

Year                                                          0
ONS Code                                                      0
LA Name                                                       0
Region                                                        0
Total Incidents                                               0
Highway Incidents                                             3
Footpath / Bridleway Incidents                                0
Back Alleyway Incidents                                       0
Railway Incidents                                             3
Council Land Incidents                                        0
Agricultural Incidents                                        0
Private / Residential Incidents                               0
Commercial / Industrial Incidents                             2
Watercourse / Bank Incidents                                  2
Other (unidentified) Incidents                                0
Animal Carcass Incidents                

In [25]:
sum_null_values(df_actions)

Year                                                                      0
ONS Code                                                                  0
LA Name                                                                   0
Region                                                                    0
Total Actions                                                             0
Investigation Actions                                                     0
Investigation Action Costs (£)                                            0
Warning Letter Actions                                                    0
Warning Letter Action Costs (£)                                           0
Statutory Notice Actions                                                  0
Statutory Notice Action Costs (£)                                         0
Fixed Penalty Notices issued specifically for fly-tipping                 0
Fixed Penalty Notices issued for Household Duty of Care                   0
Littering Fi

### Cast numeric data type on numeric columns

- From inspection some columns should have a numeric data type but do not, therefore the correct data type needs to be cast


In [26]:
df_incidents.tail(10)

,Year,ONS Code,LA Name,Region,Total Incidents,Highway Incidents,Footpath / Bridleway Incidents,Back Alleyway Incidents,Railway Incidents,Council Land Incidents,...,Transit Van Load Incidents,Transit Van Load Clearance Costs (£),Tipper Lorry Load Incidents,Tipper Lorry Load Clearance Costs (£),Significant / Multi Loads Incidents,Sig / Multi Loads Clearance Costs (£),Total Incidents Clearance Costs (£),Mid population estimates 2020,"Incidents per 1,000 people",Reporting basis
2994,2020-21,E08000015,Wirral,North West,4835,1567,473,1889,0,295,...,833,:,239,76300,848,271440,:,324336,14.9,All incidents
2995,2020-21,E07000217,Woking,South East,1705,565,0,0,0,1140,...,393,:,0,0,0,0,:,100008,17,All incidents
2996,2020-21,E06000041,Wokingham,South East,2284,1488,292,1,0,490,...,326,:,52,8100,27,14400,:,173945,13.1,All incidents
2997,2020-21,E08000031,Wolverhampton,West Midlands,3056,3056,0,0,0,0,...,179,:,39,8800,9,3240,:,264407,11.6,All incidents
2998,2020-21,E07000237,Worcester,West Midlands,432,68,115,34,0,141,...,32,:,2,300,13,1907,:,100265,4.3,All incidents
2999,2020-21,E07000229,Worthing,South East,760,552,0,0,0,16,...,36,:,2,400,0,0,:,110727,6.9,All incidents
3000,2020-21,E07000238,Wychavon,West Midlands,962,713,96,1,1,64,...,113,:,25,4250,0,0,:,131084,7.3,All incidents
3001,2020-21,E07000128,Wyre,North West,1578,105,211,1149,2,81,...,370,:,60,21000,30,10800,:,113067,14,All incidents
3002,2020-21,E07000239,Wyre Forest,West Midlands,660,273,153,28,0,118,...,205,:,1,150,13,2950,:,101139,6.5,All incidents
3003,2020-21,E06000014,York,Yorkshire and The Humber,1808,602,137,307,0,568,...,223,:,0,0,0,0,:,211012,8.6,All incidents


In [27]:
def cast_numeric_data_type(df):
    """Cast numeric data type onto columns that are not non-numeric"""
    non_numeric_columns = ("Year", "ONS Code", "LA Name", "Region", "Reporting Basis")
    
    for column in df: 
        if column not in non_numeric_columns:
            df[column] = pd.to_numeric(df[column], errors="coerce")

cast_numeric_data_type(df_incidents)
cast_numeric_data_type(df_actions)

## Questions

Region will be set to London for the following questions:

- Which kind of incidents happen most in your region?
- How many people have been sent to prison in your region?
- In which region is flytipping growing the fastest?
- Which region has the biggest fluctuations in clean costs?
- Which region is losing the most money from flytipping?

In [28]:
df_incidents

,Year,ONS Code,LA Name,Region,Total Incidents,Highway Incidents,Footpath / Bridleway Incidents,Back Alleyway Incidents,Railway Incidents,Council Land Incidents,...,Transit Van Load Incidents,Transit Van Load Clearance Costs (£),Tipper Lorry Load Incidents,Tipper Lorry Load Clearance Costs (£),Significant / Multi Loads Incidents,Sig / Multi Loads Clearance Costs (£),Total Incidents Clearance Costs (£),Mid population estimates 2020,"Incidents per 1,000 people",Reporting basis
0,2012-13,*Total,*Total,*Total England,714637.0,324284.0,96009.0,114044.0,349.0,125968.0,...,76569.0,8805444.0,11501.0,3344394.0,5202.0,1784224.0,36585739.0,NaN,NaN,NaN
1,2012-13,*Total,*Total,East Midlands,40059.0,20345.0,4808.0,1017.0,29.0,8717.0,...,3452.0,396980.0,400.0,136717.0,121.0,44975.0,1930602.0,NaN,NaN,NaN
2,2012-13,*Total,*Total,East,56484.0,20857.0,3737.0,8732.0,21.0,19750.0,...,6268.0,720820.0,805.0,212501.0,251.0,126105.0,3032202.0,NaN,NaN,NaN
3,2012-13,*Total,*Total,London,231894.0,150896.0,44594.0,5502.0,64.0,14904.0,...,19204.0,2208460.0,2000.0,680498.0,1193.0,438207.0,10803983.0,NaN,NaN,NaN
4,2012-13,*Total,*Total,North East,56748.0,5297.0,6653.0,25961.0,53.0,15435.0,...,6738.0,774870.0,1086.0,327152.0,358.0,122710.0,3136636.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2999,2020-21,E07000229,Worthing,South East,760.0,552.0,0.0,0.0,0.0,16.0,...,36.0,NaN,2.0,400.0,0.0,0.0,NaN,110727.0,6.9,NaN
3000,2020-21,E07000238,Wychavon,West Midlands,962.0,713.0,96.0,1.0,1.0,64.0,...,113.0,NaN,25.0,4250.0,0.0,0.0,NaN,131084.0,7.3,NaN
3001,2020-21,E07000128,Wyre,North West,1578.0,105.0,211.0,1149.0,2.0,81.0,...,370.0,NaN,60.0,21000.0,30.0,10800.0,NaN,113067.0,14.0,NaN
3002,2020-21,E07000239,Wyre Forest,West Midlands,660.0,273.0,153.0,28.0,0.0,118.0,...,205.0,NaN,1.0,150.0,13.0,2950.0,NaN,101139.0,6.5,NaN


#### 1) Which kinds of incidents happen most in your region?

- Set region

In [35]:
def get_incidents_for_region(df_incidents, region="London"):
    """Returns an incidents only dataframe for a specified region

    Args:
        df_incidents: dataframe of flytipping incidents
        region (str, optional): region in the UK. Defaults to "London".
    """
    incident_columns = list()

    for column in df_incidents:
        if "Incidents" in column:
            incident_columns.append(column)
        
    return df_incidents[df_incidents["Region"] == region][incident_columns]

In [36]:
df_ldn_incidents = get_incidents_for_region(df_incidents)
df_mean_ldn_incidents = df_ldn_incidents.mean(skipna=True).sort_values(ascending=False).iloc[2:]

In [37]:
df_mean_ldn_incidents

Highway Incidents                                             10813.784053
Other Household Waste Incidents                                8937.352159
Car Boot or Less Incidents                                     6721.009967
Small Van Load Incidents                                       5712.179402
Footpath / Bridleway Incidents                                 4712.893688
Black Bags - Household Incidents                               3774.913621
Single Item Incidents                                          3357.960265
Primary Waste Type Measures Other (unidentified) Incidents     2558.956811
Other (unidentified) Incidents                                 1999.768212
Transit Van Load Incidents                                     1921.443709
Council Land Incidents                                         1430.920530
Single Black Bag Incidents                                     1318.003322
Black Bags - Commercial Incidents                              1108.622517
Constr / Demol / Excav In